In [1]:
!python -m pip install mediapipe
!pip3 install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: opencv-contrib-python
    Found existing installation: opencv-contrib-python 4.12.0.88
    Uninstalling opencv

In [ ]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
import csv
from google.colab.patches import cv2_imshow
from tqdm.notebook import tqdm
from IPython.display import Video, display

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def write_landmarks_to_csv(landmarks, frame_number, csv_data):
    print(f"Landmark coordinates for frame {frame_number}:")
    for idx, landmark in enumerate(landmarks):
        print(f"{mp_pose.PoseLandmark(idx).name}: (x: {landmark.x}, y: {landmark.y}, z: {landmark.z})")
        csv_data.append([frame_number, mp_pose.PoseLandmark(idx).name, landmark.x, landmark.y, landmark.z])
    print("\n")

### Input & Output Paths


In [ ]:
in_path = '/content/drive/MyDrive/University/Deep/Sinner_2min.mp4'
out_path = '/content/drive/MyDrive/University/Deep/Sinner_2min_annoted.mp4'
csv_path = '/content/drive/MyDrive/University/Deep/Sinner_2min_landmarks.csv'

### MediaPipe Setup

In [ ]:
# Initialize MediaPipe Pose and Drawing utilities
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

### Video Capture


In [ ]:
# Open the video file
cap = cv2.VideoCapture(in_path)
if not cap.isOpened():
    print(f"Error opening video file: {in_path}")

fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

### Video Writer


In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(out_path, fourcc, fps, (w, h))
if not writer.isOpened():
    print(f"Error opening video writer: {out_path}")

### CSV


In [ ]:
write_csv = True
if write_csv:
    csv_file = open(csv_path, "w", newline="")
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["frame", "landmark", "x", "y", "z", "visibility"])

In [ ]:
def dump_landmarks_to_csv(landmarks, frame_idx):
    if not write_csv:
        return
    # MediaPipe Pose ha 33 landmark
    for i, lm in enumerate(landmarks):
        csv_writer.writerow([frame_idx, i, lm.x, lm.y, lm.z, lm.visibility])

### Video Processing

Filtraggio dei landmarks interessati, quelli che non riguardano informazioni sul viso.

In [ ]:
from mediapipe.python.solutions.pose import PoseLandmark

BODY_LANDMARKS = [
    PoseLandmark.LEFT_SHOULDER,
    PoseLandmark.RIGHT_SHOULDER,
    PoseLandmark.LEFT_ELBOW,
    PoseLandmark.RIGHT_ELBOW,
    PoseLandmark.LEFT_WRIST,
    PoseLandmark.RIGHT_WRIST,
    PoseLandmark.LEFT_HIP,
    PoseLandmark.RIGHT_HIP,
    PoseLandmark.LEFT_KNEE,
    PoseLandmark.RIGHT_KNEE,
    PoseLandmark.LEFT_ANKLE,
    PoseLandmark.RIGHT_ANKLE,
    PoseLandmark.LEFT_HEEL,
    PoseLandmark.RIGHT_HEEL,
    PoseLandmark.LEFT_FOOT_INDEX,
    PoseLandmark.RIGHT_FOOT_INDEX,
]

In [ ]:
def filtered_landmarks(landmarks):
  out = {}
  for name,

In [ ]:
from tqdm import tqdm

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_idx = 0

# Usa il model_complexity=1 (bilancia velocità/accuratezza). Se hai GPU/tempo, metti 2.
with mp_pose.Pose(static_image_mode=False, model_complexity=1) as pose:
  with tqdm(total=total_frames, desc="Video elaboration", unit="frame") as pbar:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # BGR -> RGB per Mediapipe
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = pose.process(rgb)

        if result.pose_landmarks:
            # Disegna sul frame BGR (ok)
            mp_drawing.draw_landmarks(
                frame,
                result.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing.DrawingSpec(thickness=2, circle_radius=2),
                connection_drawing_spec=mp_drawing.DrawingSpec(thickness=2)
            )
            # Salva anche i landmark su CSV (opzionale)
            dump_landmarks_to_csv(result.pose_landmarks.landmark, frame_idx)

        # Scrivi il frame annotato nel video
        writer.write(frame)
        frame_idx += 1
        pbar.update(1)

# Cleanup
cap.release()
writer.release()
if write_csv:
    csv_file.close()

print(f"Video saved in: {out_path}")
if write_csv:
    print(f"CSV saved in: {csv_path}")

Video elaboration: 0frame [00:00, ?frame/s]

Video saved in: /content/drive/MyDrive/University/Deep/Sinner_2min_annoted.mp4
CSV saved in: /content/drive/MyDrive/University/Deep/Sinner_2min_landmarks.csv


In [ ]:
display(Video(out_path, embed=True))

Output hidden; open in https://colab.research.google.com to view.